# Missions Editor

Generate and edit route deck missions, then save back to `board.json`.


In [1]:
import json
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

board = json.loads(Path("board.json").read_text())
places = {p["id"]: p for p in board.get("places", [])}
roads = board.get("roads", [])


routes_md_path = Path("routes.md")
routes_md_text = routes_md_path.read_text().strip() if routes_md_path.exists() else ""


In [2]:
# Read current missions (standard and highways)
route_decks = board.get("routeDecks", {})
place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}

def format_mission(mission, idx=None):
    ids = mission.get("placeIds", [])
    if len(ids) != 2:
        label = "(invalid)"
    else:
        a, b = ids
        label = f"{place_name.get(a, a)} — {place_name.get(b, b)}"
    score = mission.get("score", "?")
    prefix = f"{idx:02d}. " if idx is not None else ""
    return f"{prefix}{label} (score: {score})"

for deck_name in ["standard", "highways"]:
    missions = route_decks.get(deck_name, [])
    print(f"{deck_name.upper()} MISSIONS (current)")
    if not missions:
        print("(none)")
        continue
    for idx, mission in enumerate(missions, 1):
        print(format_mission(mission, idx))


# Delete all missions
delete_all_missions = widgets.Button(description="Delete All Missions", button_style="danger")

def handle_delete_all(_):
    board["routeDecks"]["standard"] = []
    board["routeDecks"]["highways"] = []
    print("Deleted all missions (in-memory). Save to persist.")

delete_all_missions.on_click(handle_delete_all)
display(delete_all_missions)


STANDARD MISSIONS (current)
01. Washington DC — Mandalay (score: 10)
02. Tashkent — Yangon (score: 6)
03. Hilo — Goesan (score: 6)
04. Columbus — Daegu (score: 10)
05. Tashkent — Beijing (score: 5)
06. Izmir — Beijing (score: 9)
07. Long Beach — Busan (score: 5)
08. Philadelphia — Yangon (score: 8)
09. Riverside — Dili (score: 7)
10. Riverside — Istanbul (score: 10)
11. Almaty — Mandalay (score: 5)
12. Riverside — Shanghai (score: 7)
13. Riverside — Philadelphia (score: 7)
14. Long Beach — Shanghai (score: 6)
15. Long Beach — Boston (score: 9)
16. Berkeley — Seoul (score: 7)
17. Goesan — Tashkent (score: 7)
18. Moscow — Shanghai (score: 10)
19. Daegu — Moscow (score: 10)
20. Berkeley — Yangon (score: 10)
21. Long Beach — Washington DC (score: 7)
22. Seoul — Tashkent (score: 6)
23. Goesan — Tbilisi (score: 9)
24. Hilo — Mawlamyine (score: 8)
25. Almaty — Dili (score: 8)
26. Fairbanks — Yerevan (score: 10)
27. Berkeley — Boston (score: 10)
28. Long Beach — Philadelphia (score: 9)
29. Bus

Button(button_style='danger', description='Delete All Missions', style=ButtonStyle())

In [3]:
# Route input, color balancing, and stats
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display, clear_output

place_name_to_id = {p.get("name", str(pid)).lower(): pid for pid, p in places.items()}

place_options = sorted([(p.get("name", str(pid)), pid) for pid, p in places.items()], key=lambda x: x[0])


def lookup_place_id(name):
    if name is None:
        return None
    name = str(name).strip()
    if not name:
        return None
    if name.isdigit():
        pid = int(name)
        return pid if pid in places else None
    pid = place_name_to_id.get(name.lower())
    if pid is not None:
        return pid
    # Try case-insensitive match with stripped punctuation spacing
    key = " ".join(name.lower().split())
    return place_name_to_id.get(key)


def route_length_for_missions(route):
    length = float(route.get("spaceAmount", 0) or 0)
    if route.get("routeType") == "ocean":
        length *= 0.5
    if route.get("tunnelCostDouble"):
        length *= 2
    return length


def _split_city_pair(pair):
    for sep in ["-"]:
        if sep in pair:
            a, b = pair.split(sep, 1)
            return a.strip(), b.strip()
    return None, None


def parse_route_line(line):
    line = line.strip()
    if not line or line.startswith("#"):
        return None
    tokens = line.split()
    if len(tokens) < 4:
        raise ValueError("Expected: CityA-CityB length color type")
    length_token, color_token, type_token = tokens[-3:]
    pair = " ".join(tokens[:-3]).strip()
    a_name, b_name = _split_city_pair(pair)
    if not a_name or not b_name:
        raise ValueError("City pair must use a dash: CityA-CityB")
    a_id = lookup_place_id(a_name)
    b_id = lookup_place_id(b_name)
    if a_id is None or b_id is None:
        raise ValueError(f"Unknown city in: {a_name} / {b_name}")
    try:
        length = float(length_token)
    except ValueError:
        raise ValueError(f"Invalid length: {length_token}")
    color = color_token.strip().lower()
    if color == "gray":
        color = "grey"
    rtype = type_token.strip().lower()
    if rtype not in {"land", "ocean"}:
        raise ValueError("Route type must be land or ocean")
    return {
        "placeIds": [a_id, b_id],
        "spaceAmount": length,
        "colour": color,
        "routeType": rtype,
    }


def write_board_json():
    Path("board.json").write_text(json.dumps(board, ensure_ascii=False, indent=2))


def routes_to_md_lines():
    lines = []
    for r in roads:
        ids = r.get("placeIds", [])
        if len(ids) != 2:
            continue
        a, b = ids
        a_name = places.get(a, {}).get("name", str(a))
        b_name = places.get(b, {}).get("name", str(b))
        length = r.get("spaceAmount", 0) or 0
        color = "grey"
        lanes = r.get("lanes", [])
        if lanes:
            color = lanes[0].get("colour", color)
        rtype = r.get("routeType", "land")
        lines.append(f"{a_name}-{b_name} {length} {color} {rtype}")
    return lines


def write_routes_md():
    routes_md_path = Path("routes.md")
    lines = routes_to_md_lines()
    routes_md_path.write_text("\n".join(lines))


def _next_ids(existing_roads):
    max_road_id = 0
    max_lane_id = 0
    for r in existing_roads:
        max_road_id = max(max_road_id, int(r.get("id", 0) or 0))
        for lane in r.get("lanes", []):
            max_lane_id = max(max_lane_id, int(lane.get("id", 0) or 0))
    return max_road_id, max_lane_id


def apply_routes(parsed_routes, replace_existing=False):
    global roads
    if replace_existing:
        roads = []
    max_road_id, max_lane_id = _next_ids(roads)
    for r in parsed_routes:
        max_road_id += 1
        max_lane_id += 1
        road = {
            "id": max_road_id,
            "placeIds": r["placeIds"],
            "spaceAmount": r["spaceAmount"],
            "lanes": [{"id": max_lane_id, "colour": r["colour"]}],
        }
        if r["routeType"] == "ocean":
            road["routeType"] = "ocean"
        roads.append(road)
    board["roads"] = roads


def compute_stats(use_mission_lengths=False):
    adj = {pid: set() for pid in places}
    edge_lengths = []
    color_totals = defaultdict(float)
    for r in roads:
        ids = r.get("placeIds", [])
        if len(ids) != 2:
            continue
        a, b = ids
        adj.setdefault(a, set()).add(b)
        adj.setdefault(b, set()).add(a)
        length = route_length_for_missions(r) if use_mission_lengths else float(r.get("spaceAmount", 0) or 0)
        if (not use_mission_lengths) and r.get("tunnelCostDouble"):
            length *= 2
        if length > 0:
            edge_lengths.append(length)
        for lane in r.get("lanes", []):
            color = lane.get("colour", "grey").lower()
            if color == "gray":
                color = "grey"
            color_totals[color] += length
    degrees = {pid: len(neigh) for pid, neigh in adj.items()}
    return degrees, edge_lengths, color_totals


def render_stats(use_mission_lengths=False):
    degrees, edge_lengths, color_totals = compute_stats(use_mission_lengths)
    with stats_output:
        clear_output()
        print("City degree distribution (count -> cities):")
        degree_counts = defaultdict(int)
        for d in degrees.values():
            degree_counts[d] += 1
        for d in sorted(degree_counts):
            print(f"  {d}: {degree_counts[d]}")

        print("Route length distribution:")
        length_counts = defaultdict(int)
        for w in edge_lengths:
            length_counts[w] += 1
        for w in sorted(length_counts):
            print(f"  {w}: {length_counts[w]}")

        print("Color totals (by length):")
        for c in sorted(color_totals):
            print(f"  {c}: {round(color_totals[c], 2)}")


def balance_colors_even_except_grey(allow_recolor_non_grey=False):
    target_colors = ["red", "yellow", "purple", "green", "black", "white"]

    def lane_length(route):
        length = float(route.get("spaceAmount", 0) or 0)
        if route.get("tunnelCostDouble"):
            length *= 2
        return length

    # Current totals
    totals = {c: 0.0 for c in target_colors}
    for r in roads:
        length = lane_length(r)
        for lane in r.get("lanes", []):
            color = lane.get("colour", "grey").lower()
            if color == "gray":
                color = "grey"
            if color in totals:
                totals[color] += length

    total_len = sum(totals.values())
    if total_len <= 0:
        return
    target = total_len / len(target_colors)

    # Candidates (prefer grey unless allowed)
    candidates = []
    for r in roads:
        for lane in r.get("lanes", []):
            color = lane.get("colour", "grey").lower()
            if color == "gray":
                color = "grey"
            if color == "grey" or allow_recolor_non_grey:
                candidates.append((r, lane, color))

    candidates.sort(key=lambda item: lane_length(item[0]), reverse=True)

    # Greedy fill under-target colors
    for target_color in target_colors:
        need = max(0.0, target - totals.get(target_color, 0.0))
        if need <= 0:
            continue
        for r, lane, current_color in list(candidates):
            if need <= 0:
                break
            if current_color == target_color:
                continue
            if not allow_recolor_non_grey and current_color != "grey":
                continue
            length = lane_length(r)
            lane["colour"] = target_color
            if current_color in totals:
                totals[current_color] -= length
            totals[target_color] += length
            need -= length

    board["roads"] = roads


def parse_color_targets(text):
    targets = {}
    for line in text.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        parts = line.split()
        if len(parts) != 2:
            raise ValueError("Color targets must be: color count")
        color, count = parts
        color = color.lower()
        if color == "gray":
            color = "grey"
        targets[color] = float(count)
    return targets


def adjust_color_distribution(targets, allow_recolor_non_grey=False):
    # Greedy recolor by lane length, preferring grey
    color_totals = defaultdict(float)
    for r in roads:
        length = float(r.get("spaceAmount", 0) or 0)
        if r.get("tunnelCostDouble"):
            length *= 2
        for lane in r.get("lanes", []):
            color = lane.get("colour", "grey").lower()
            if color == "gray":
                color = "grey"
            color_totals[color] += length

    def lane_length(route):
        length = float(route.get("spaceAmount", 0) or 0)
        if route.get("tunnelCostDouble"):
            length *= 2
        return length

    # Gather candidate lanes
    candidates = []
    for r in roads:
        for lane in r.get("lanes", []):
            color = lane.get("colour", "grey").lower()
            if color == "gray":
                color = "grey"
            if color == "grey" or allow_recolor_non_grey:
                candidates.append((r, lane, color))

    # Sort candidates by length descending for better matching
    candidates.sort(key=lambda item: lane_length(item[0]), reverse=True)

    for target_color, target_total in targets.items():
        current = color_totals.get(target_color, 0.0)
        if current >= target_total:
            continue
        needed = target_total - current
        for r, lane, current_color in list(candidates):
            if needed <= 0:
                break
            if current_color == target_color:
                continue
            length = lane_length(r)
            if not allow_recolor_non_grey and current_color != "grey":
                continue
            lane["colour"] = target_color
            color_totals[current_color] -= length
            color_totals[target_color] += length
            needed -= length

    board["roads"] = roads


# Widgets
routes_input = widgets.Textarea(
    description="Routes",
    layout=widgets.Layout(width="100%", height="160px"),
    placeholder=(
        "One route per line. Format: CityA-CityB length color type\n"
        "Example: Paris-Berlin 4 red land\n"
        "Example: Lisboa-Cadiz 3 blue ocean"
    ),
)
replace_existing = widgets.Checkbox(description="Replace existing roads", value=False)

route_color_options = ["red", "yellow", "purple", "green", "black", "white", "gray"]
route_length_options = list(range(1, 10))
route_type_options = ["land", "ocean"]

route_city_a = widgets.Dropdown(options=place_options, description="City A")
route_city_b = widgets.Dropdown(options=place_options, description="City B")
route_length = widgets.Dropdown(options=route_length_options, value=4, description="Length")
route_color = widgets.Dropdown(options=route_color_options, value="gray", description="Color")
route_type = widgets.Dropdown(options=route_type_options, value="land", description="Type")
add_line_btn = widgets.Button(description="Add Route Line", button_style="primary")
add_apply_btn = widgets.Button(description="Add + Apply", button_style="success")
if routes_md_text:
    routes_input.value = routes_md_text
preview_btn = widgets.Button(description="Preview Parse", button_style="info")
apply_btn = widgets.Button(description="Apply Routes", button_style="primary")
clear_routes_btn = widgets.Button(description="Clear All Routes", button_style="danger")
parse_output = widgets.Output()

color_targets = widgets.Textarea(
    description="Color targets",
    layout=widgets.Layout(width="100%", height="120px"),
    placeholder="One per line: color count (e.g. red 20)",
)
recolor_non_grey = widgets.Checkbox(description="Allow recolor non-grey", value=False)
adjust_colors_btn = widgets.Button(description="Adjust Color Distribution", button_style="warning")

even_colors_btn = widgets.Button(description="Even Colors (no grey)", button_style="warning")
color_output = widgets.Output()

stats_mode = widgets.ToggleButtons(options=[("Raw", "raw"), ("Mission (ocean /2)", "mission")], value="raw", description="Lengths")
refresh_stats_btn = widgets.Button(description="Refresh Stats", button_style="success")
stats_output = widgets.Output()


def handle_add_line(_):
    a = places[route_city_a.value]["name"]
    b = places[route_city_b.value]["name"]
    if route_city_a.value == route_city_b.value:
        with parse_output:
            clear_output()
            print("Error: City A and City B must differ.")
        return
    line = f"{a}-{b} {route_length.value} {route_color.value} {route_type.value}"
    existing = routes_input.value.strip()
    routes_input.value = (existing + "\n" if existing else "") + line


def handle_add_apply(_):
    handle_add_line(None)
    handle_apply(None)


def handle_preview(_):
    with parse_output:
        clear_output()
        try:
            parsed = [parse_route_line(line) for line in routes_input.value.splitlines()]
            parsed = [p for p in parsed if p is not None]
            print(f"Parsed {len(parsed)} routes:")
            for p in parsed:
                a, b = p["placeIds"]
                print(f"  {places[a]['name']} - {places[b]['name']} | {p['spaceAmount']} | {p['colour']} | {p['routeType']}")
        except Exception as e:
            print(f"Error: {e}")


def handle_apply(_):
    with parse_output:
        clear_output()
        try:
            parsed = [parse_route_line(line) for line in routes_input.value.splitlines()]
            parsed = [p for p in parsed if p is not None]
            apply_routes(parsed, replace_existing=replace_existing.value)
            print(f"Applied {len(parsed)} routes. Total roads: {len(roads)}")
        except Exception as e:
            print(f"Error: {e}")


def handle_clear_routes(_):
    global roads
    with parse_output:
        clear_output()
        roads = []
        board["roads"] = roads
        print("Cleared all routes. Total roads: 0")



def handle_even_colors(_):
    with color_output:
        clear_output()
        try:
            balance_colors_even_except_grey(allow_recolor_non_grey=recolor_non_grey.value)
            write_routes_md()
            write_board_json()
            print("Balanced colors (grey excluded).")
        except Exception as e:
            print(f"Error: {e}")


def handle_adjust_colors(_):
    with color_output:
        clear_output()
        try:
            targets = parse_color_targets(color_targets.value)
            adjust_color_distribution(targets, allow_recolor_non_grey=recolor_non_grey.value)
            write_routes_md()
            write_board_json()
            print("Color distribution adjusted.")
        except Exception as e:
            print(f"Error: {e}")


def handle_refresh_stats(_):
    use_mission = stats_mode.value == "mission"
    render_stats(use_mission_lengths=use_mission)


preview_btn.on_click(handle_preview)
apply_btn.on_click(handle_apply)
clear_routes_btn.on_click(handle_clear_routes)
add_line_btn.on_click(handle_add_line)
add_apply_btn.on_click(handle_add_apply)
even_colors_btn.on_click(handle_even_colors)
adjust_colors_btn.on_click(handle_adjust_colors)
refresh_stats_btn.on_click(handle_refresh_stats)


ui_builder = widgets.VBox([
    widgets.HTML("<b>Route Builder</b>"),
    widgets.HBox([route_city_a, route_city_b]),
    widgets.HBox([route_length, route_color, route_type]),
    widgets.HBox([add_line_btn, add_apply_btn]),
])

ui_routes = widgets.VBox([
    widgets.HTML("<b>Route Input</b>"),
    routes_input,
    widgets.HBox([preview_btn, apply_btn, clear_routes_btn, replace_existing]),
    parse_output,
])

ui_colors = widgets.VBox([
    widgets.HTML("<b>Color Distribution Adjustment</b>"),
    color_targets,
    widgets.HBox([even_colors_btn, adjust_colors_btn, recolor_non_grey]),
    color_output,
])

ui_stats = widgets.VBox([
    widgets.HTML("<b>Current Route Stats</b>"),
    widgets.HBox([stats_mode, refresh_stats_btn]),
    stats_output,
])

handle_refresh_stats(None)

display(ui_builder, ui_routes, ui_colors, ui_stats)


In [4]:
# Redesign missions: short standard, long highways (difficulty-aware)
import random
import heapq
import ipywidgets as widgets
from IPython.display import display

random.seed(7)

# Build weighted graph with tunnel cost doubling
weighted_adj = {pid: [] for pid in places}
for r in roads:
    ids = r.get("placeIds", [])
    if len(ids) != 2:
        continue
    a, b = ids
    if 'route_length_for_missions' in globals():
        w = route_length_for_missions(r)
    else:
        w = float(r.get("spaceAmount", 0) or 0)
        if r.get("tunnelCostDouble"):
            w *= 2
    weighted_adj[a].append((b, w))
    weighted_adj[b].append((a, w))


def shortest_path_cost_and_hops(src, dst):
    # Dijkstra to get cost and hops count
    pq = [(0, 0, src)]  # cost, hops, node
    seen = {}
    while pq:
        cost, hops, node = heapq.heappop(pq)
        if node in seen:
            continue
        seen[node] = (cost, hops)
        if node == dst:
            return cost, hops
        for nxt, w in weighted_adj.get(node, []):
            if nxt in seen:
                continue
            heapq.heappush(pq, (cost + w, hops + 1, nxt))
    return None, None


def pick_pairs_by_hops(hops_min, hops_max, target_count, prefer_hops):
    pairs = []
    ids = list(places.keys())
    # Precompute candidate pairs
    candidates = []
    for i in range(len(ids)):
        for j in range(i + 1, len(ids)):
            a, b = ids[i], ids[j]
            cost, hops = shortest_path_cost_and_hops(a, b)
            if cost is None:
                continue
            if hops_min <= hops <= hops_max:
                candidates.append((a, b, cost, hops))
    if not candidates:
        return []

    # Weight by closeness to preferred hop count
    def weight(h):
        return 1.0 / (1 + abs(h - prefer_hops))

    random.shuffle(candidates)
    while candidates and len(pairs) < target_count:
        weights = [weight(c[3]) for c in candidates]
        pick = random.choices(candidates, weights=weights, k=1)[0]
        pairs.append(pick)
        candidates.remove(pick)

    return pairs


def score_from_cost(cost, min_score, max_score, costs_sorted=None):
    # Map cost percentile to score range for smoother distribution
    if costs_sorted:
        # Percentile rank
        import bisect
        rank = bisect.bisect_left(costs_sorted, cost)
        t = rank / max(1, len(costs_sorted) - 1)
    else:
        cost = max(1, min(cost, 40))
        t = (cost - 1) / 39.0
    score = min_score + t * (max_score - min_score)
    return int(round(score))


def generate_missions():
    # Standard: 2-5 hops, prefer 3-4, score 7-14
    standard_pairs = pick_pairs_by_hops(2, 5, int(standard_count.value), prefer_hops=3.5)
    # Highways: 5-9 hops, prefer 7, score 15-28
    highway_pairs = pick_pairs_by_hops(5, 9, int(highway_count.value), prefer_hops=7)

    standard = []
    standard_costs = sorted([c for _, _, c, _ in standard_pairs])
    for a, b, cost, hops in standard_pairs:
        standard.append({
            "placeIds": [a, b],
            "score": score_from_cost(cost, 5, 11, standard_costs),
            "hops": hops,
        })

    highways = []
    highway_costs = sorted([c for _, _, c, _ in highway_pairs])
    for a, b, cost, hops in highway_pairs:
        highways.append({
            "placeIds": [a, b],
            "score": score_from_cost(cost, 15, 25, highway_costs),
            "hops": hops,
        })

    board.setdefault("routeDecks", {})
    existing_standard = board["routeDecks"].get("standard", [])
    existing_highways = board["routeDecks"].get("highways", [])
    existing_standard.extend(standard)
    existing_highways.extend(highways)
    board["routeDecks"]["standard"] = existing_standard
    board["routeDecks"]["highways"] = existing_highways


def print_sample(deck_name, limit=10):
    missions = board.get("routeDecks", {}).get(deck_name, [])
    place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}
    print(f"{deck_name.upper()} MISSIONS (sample)")
    for i, m in enumerate(missions[:limit], 1):
        a, b = m.get("placeIds", [None, None])
        print(f"{i:02d}. {place_name.get(a, a)} — {place_name.get(b, b)} (score: {m.get('score')}, hops: {m.get('hops')})")


standard_count = widgets.IntSlider(description="Standard", min=5, max=60, value=30)
highway_count = widgets.IntSlider(description="Highways", min=5, max=60, value=20)

btn_generate = widgets.Button(description="Generate Missions", button_style="primary")
btn_save = widgets.Button(description="Save Missions to board.json", button_style="success")
output = widgets.Output()


def handle_generate(_):
    generate_missions()
    with output:
        output.clear_output()


def handle_save(_):
    Path("board.json").write_text(json.dumps(board, indent=2))
    with output:
        print("Saved redesigned missions to board.json")

btn_generate.on_click(handle_generate)
btn_save.on_click(handle_save)

display(widgets.HBox([standard_count, highway_count, btn_generate, btn_save]), output)


Output()

In [5]:
# Mission editor (interactive)
import ipywidgets as widgets
from IPython.display import display

# Ensure weighted graph helpers exist
if "weighted_adj" not in globals():
    weighted_adj = {pid: [] for pid in places}
    for r in roads:
        ids = r.get("placeIds", [])
        if len(ids) != 2:
            continue
        a, b = ids
        if 'route_length_for_missions' in globals():
            w = route_length_for_missions(r)
        else:
            w = float(r.get("spaceAmount", 0) or 0)
            if r.get("tunnelCostDouble"):
                w *= 2
        weighted_adj[a].append((b, w))
        weighted_adj[b].append((a, w))

if "shortest_path_cost_and_hops" not in globals():
    import heapq

    def shortest_path_cost_and_hops(src, dst):
        pq = [(0, 0, src)]
        seen = {}
        while pq:
            cost, hops, node = heapq.heappop(pq)
            if node in seen:
                continue
            seen[node] = (cost, hops)
            if node == dst:
                return cost, hops
            for nxt, w in weighted_adj.get(node, []):
                if nxt in seen:
                    continue
                heapq.heappush(pq, (cost + w, hops + 1, nxt))
        return None, None

route_decks = board.setdefault("routeDecks", {"standard": [], "highways": []})

place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}
place_options = sorted([(p.get("name", str(pid)), pid) for pid, p in places.items()], key=lambda x: x[0])


def format_mission(mission, idx=None):
    ids = mission.get("placeIds", [])
    if len(ids) != 2:
        label = "(invalid)"
    else:
        a, b = ids
        label = f"{place_name.get(a, a)} — {place_name.get(b, b)}"
    score = mission.get("score", "?")
    prefix = f"{idx:02d}. " if idx is not None else ""
    return f"{prefix}{label} (score: {score})"


def deck_options(deck_name):
    missions = route_decks.get(deck_name, [])
    return [(format_mission(m, i + 1), i) for i, m in enumerate(missions)]


# UI widgets
deck_select = widgets.ToggleButtons(options=["standard", "highways"], value="standard", description="Deck")
mission_select = widgets.Select(options=deck_options("standard"), description="Missions", rows=8)

city_a = widgets.Dropdown(options=place_options, description="City A")
city_b = widgets.Dropdown(options=place_options, description="City B")
score = widgets.IntSlider(description="Score", min=5, max=30, value=10)

hops_label = widgets.HTML(value="<b>Hops:</b> -")
cost_label = widgets.HTML(value="<b>Min Cost:</b> -")

add_btn = widgets.Button(description="Add New", button_style="primary")
update_btn = widgets.Button(description="Update Selected", button_style="info")
remove_btn = widgets.Button(description="Remove Selected", button_style="danger")
output = widgets.Output()


def refresh_mission_list():
    mission_select.options = deck_options(deck_select.value)


def update_metrics():
    a = city_a.value
    b = city_b.value
    if a == b:
        hops_label.value = "<b>Hops:</b> 0"
        cost_label.value = "<b>Min Cost:</b> 0"
        return
    cost, hops = shortest_path_cost_and_hops(a, b)
    hops_label.value = f"<b>Hops:</b> {hops if hops is not None else 'N/A'}"
    cost_label.value = f"<b>Min Cost:</b> {cost if cost is not None else 'N/A'}"


def handle_deck_change(change):
    refresh_mission_list()


def handle_mission_select(change):
    if mission_select.value is None:
        return
    idx = mission_select.value
    missions = route_decks.get(deck_select.value, [])
    if idx < 0 or idx >= len(missions):
        return
    m = missions[idx]
    ids = m.get("placeIds", [])
    if len(ids) == 2:
        city_a.value = ids[0]
        city_b.value = ids[1]
    score.value = int(m.get("score", score.value) or score.value)
    update_metrics()


def handle_add(_):
    missions = route_decks.get(deck_select.value, [])
    missions.append({"placeIds": [city_a.value, city_b.value], "score": int(score.value)})
    route_decks[deck_select.value] = missions
    refresh_mission_list()


def handle_update(_):
    idx = mission_select.value
    if idx is None:
        return
    missions = route_decks.get(deck_select.value, [])
    if idx < 0 or idx >= len(missions):
        return
    missions[idx] = {"placeIds": [city_a.value, city_b.value], "score": int(score.value)}
    route_decks[deck_select.value] = missions
    refresh_mission_list()


def handle_remove(_):
    idx = mission_select.value
    if idx is None:
        return
    missions = route_decks.get(deck_select.value, [])
    if idx < 0 or idx >= len(missions):
        return
    missions.pop(idx)
    route_decks[deck_select.value] = missions
    refresh_mission_list()



deck_select.observe(handle_deck_change, names="value")
mission_select.observe(handle_mission_select, names="value")
city_a.observe(lambda change: update_metrics(), names="value")
city_b.observe(lambda change: update_metrics(), names="value")

add_btn.on_click(handle_add)
update_btn.on_click(handle_update)
remove_btn.on_click(handle_remove)

refresh_mission_list()
update_metrics()

ui = widgets.VBox([
    deck_select,
    mission_select,
    widgets.HBox([city_a, city_b]),
    widgets.HBox([score, hops_label, cost_label]),
    widgets.HBox([add_btn, update_btn, remove_btn]),
])

display(ui, output)


Output()

In [6]:
# Deduplicate missions and save
import ipywidgets as widgets
from IPython.display import display

remove_dupes = widgets.Button(description="Remove Duplicate Missions", button_style="warning")
save_all = widgets.Button(description="Save Missions to board.json", button_style="success")
output_check = widgets.Output()


def normalize_key(m):
    ids = m.get("placeIds", [])
    if len(ids) != 2:
        return None
    return tuple(sorted(ids))


def handle_remove_dupes(_):
    for deck in ["standard", "highways"]:
        missions = board.get("routeDecks", {}).get(deck, [])
        seen = set()
        unique = []
        for m in missions:
            key = normalize_key(m)
            if key is None:
                continue
            if key in seen:
                continue
            seen.add(key)
            unique.append(m)
        board["routeDecks"][deck] = unique
    with output_check:
        output_check.clear_output()
        print("Removed duplicate missions.")


def handle_save_all(_):
    Path("board.json").write_text(json.dumps(board, indent=2))
    with output_check:
        print("Saved missions to board.json")

remove_dupes.on_click(handle_remove_dupes)
save_all.on_click(handle_save_all)

display(widgets.HBox([remove_dupes, save_all]), output_check)


Output()

In [7]:
# Export missions to Markdown
from pathlib import Path

place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}

lines = ["# Missions", ""]
for deck_name in ["standard", "highways"]:
    missions = board.get("routeDecks", {}).get(deck_name, [])
    lines.append(f"## {deck_name.title()}")
    if not missions:
        lines.append("(none)")
        lines.append("")
        continue
    for m in missions:
        ids = m.get("placeIds", [])
        if len(ids) != 2:
            continue
        a, b = ids
        score = m.get("score", "?")
        lines.append(f"- {place_name.get(a, a)} — {place_name.get(b, b)} (score: {score})")
    lines.append("")

Path("missions.md").write_text("\n".join(lines))
print("Wrote missions.md")


Wrote missions.md


In [8]:
# Export missions to DOCX (with category annotations)
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    print("python-docx is not installed. Install it to enable DOCX export.")
else:
    place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}

    doc = Document()
    doc.add_heading("Ticket To Ride Missions", level=1)

    doc.add_paragraph(
        "Standard missions are shorter and lower score (5–11). Highways are longer and higher score (15–25)."
    )

    def add_mission_table(missions, label):
        # Two-column table with small category label
        table = doc.add_table(rows=0, cols=2)
        for i, m in enumerate(missions):
            ids = m.get("placeIds", [])
            if len(ids) != 2:
                continue
            a, b = ids
            score = m.get("score", "?")
            text = f"{place_name.get(a, a)} — {place_name.get(b, b)} (score: {score})"
            if i % 2 == 0:
                row = table.add_row()
                cell = row.cells[0]
            else:
                cell = row.cells[1]
            p = cell.paragraphs[0]
            run_label = p.add_run(f"[{label}] ")
            run_label.font.size = Pt(8)
            run_label.bold = True
            run_text = p.add_run(text)
            run_text.font.size = Pt(10)
        return table

    for deck_name, note, label in [
        ("standard", "Short routes, lower scores.", "Standard"),
        ("highways", "Longer routes, higher scores.", "Highway"),
    ]:
        missions = board.get("routeDecks", {}).get(deck_name, [])
        doc.add_heading(deck_name.title(), level=2)
        doc.add_paragraph(note)
        if not missions:
            doc.add_paragraph("(none)")
            continue
        add_mission_table(missions, label)

    doc.save("missions.docx")
    print("Wrote missions.docx")


Wrote missions.docx


In [16]:
# Export missions to PDF (8 cards per A4 page)
from pathlib import Path
import math

try:
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import A4
    from reportlab.lib.units import mm
    from reportlab.lib import colors
except ImportError:
    print("reportlab is not installed. Install it to enable PDF export (pip install reportlab).")
else:
    place_name = {pid: p.get("name", str(pid)) for pid, p in places.items()}
    out_path = Path("missions_cards.pdf")

    # Layout: 2 columns x 4 rows, evenly spaced on A4
    page_w, page_h = A4
    cols, rows = 2, 4
    margin_x = 12 * mm
    margin_y = 12 * mm
    gap_x = 8 * mm
    gap_y = 8 * mm
    card_w = (page_w - 2 * margin_x - (cols - 1) * gap_x) / cols
    card_h = (page_h - 2 * margin_y - (rows - 1) * gap_y) / rows

    c = canvas.Canvas(str(out_path), pagesize=A4)
    c.setAuthor("TTR Stats")
    c.setTitle("Ticket To Ride Missions")

    def draw_card(x, y, route_text, points_text, footer=None, fill_color=colors.whitesmoke):
        radius = 6 * mm
        c.setStrokeColor(colors.black)
        c.setLineWidth(1)
        c.setFillColor(fill_color)
        c.roundRect(x, y, card_w, card_h, radius, stroke=1, fill=1)

        pad = 6 * mm
        text_x = x + pad
        text_y = y + card_h - pad

        c.setFillColor(colors.black)
        c.setFont("Helvetica-Bold", 16)
        c.drawString(text_x, text_y - 12, route_text)

        # Bottom-right points
        c.setFont("Helvetica-Bold", 32)
        c.drawRightString(x + card_w - pad, y + pad, points_text)

        if footer:
            c.setFont("Helvetica-Oblique", 8)
            c.drawString(text_x, y + pad, footer)

    def mission_lines(m):
        a, b = m.get("placeIds", [None, None])
        a_name = place_name.get(a, str(a))
        b_name = place_name.get(b, str(b))
        score = m.get("score", m.get("points", ""))
        return f"{a_name}  →  {b_name}", f"{score}"

    missions_all = []
    for deck_name in ["standard", "highways"]:
        missions = board.get("routeDecks", {}).get(deck_name, [])
        for m in missions:
            missions_all.append((deck_name, m))

    deck_fill = {
        "standard": colors.whitesmoke,
        "highways": colors.lightgrey,
    }
    if not missions_all:
        print("No missions found.")
    else:
        per_page = cols * rows
        for idx, (deck_name, m) in enumerate(missions_all):
            slot = idx % per_page
            if slot == 0 and idx != 0:
                c.showPage()
            col = slot % cols
            row = rows - 1 - (slot // cols)
            x = margin_x + col * (card_w + gap_x)
            y = margin_y + row * (card_h + gap_y)
            route_text, points_text = mission_lines(m)
            fill = deck_fill.get(deck_name, colors.whitesmoke)
            draw_card(x, y, route_text, points_text, footer=deck_name.title(), fill_color=fill)

        c.save()
        print(f"Wrote {out_path}")


Wrote missions_cards.pdf
